# Lab Assignment 2

- Name - Aryan Gupta
- Roll No.- 230150003
- Date - 6 Aug, 2025
- Course - DA331 Big Data Analytics: Tools & Techniques

## Import Library

In [144]:
# Install Spark
!pip install pyspark
# !pip install pyspark[sql]

In [145]:
# import pyspark.sql classes and functions
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col, array_contains, isnan, when, count
from pyspark.sql.functions import lit, concat_ws, concat, collect_list, udf
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import max, corr

# time library
import time

# pandas
import pandas as pd

spark = SparkSession.builder.appName("answers").getOrCreate()

## Load Data

In [146]:
# Get where the session is built
!pwd

/content


In [147]:
# Define path
path = "./sample_data/california_housing_train.csv"

### Get the feel of the data

In [148]:
df = spark.read.option("header",'True').option('delimiter', ',').csv(path)
df.show()
df.printSchema()

+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+-----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000| 472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000| 463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000| 117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000| 226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000| 262.000000|     1.925000|    

### Load actual schema

In [149]:
# # define schema
# schema = StructType() \
#       .add("longitude",DoubleType(),True) \
#       .add("latitude",DoubleType(),True) \
#       .add("housing_median_age",DoubleType(),True) \
#       .add("total_rooms",DoubleType(),True) \
#       .add("total_bedrooms",DoubleType(),True) \
#       .add("population",DoubleType(),True) \
#       .add("households",DoubleType(),True) \
#       .add("median_income",DoubleType(),True) \
#       .add("median_house_value",DoubleType(),True) \

In [150]:
# df = spark.read.format("csv") \
#       .option("header", True) \
#       .option('delimiter', ',') \
#       .schema(schema) \
#       .load("/content/sample_data/california_housing_train.csv")

# df.printSchema()

### OR change datatype of the columns

In [151]:
columns = df.columns
print(columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']


In [152]:
for one_col in columns:
  df = df.withColumn(one_col, col(one_col).cast(DoubleType()))
df.printSchema()
df.show(5)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|     

## Show Summary

In [153]:
df.summary().show()

+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|             17000|            17000|            17000|             17000|            17000|             17000|             17000|
|   mean|-119.56210823529375|  35.6252247058827| 28.58935294117647|2643.664411764706|539.4108235294118|1429.5739411764705|501.2219411764706| 3.883578100000021|207300.91235294117|
| stddev| 2.0051664084260357|2.1373397946570867|12.586936981660406|2179.947071452777|421.4994515798648| 1

## 1.) Average number of household for different geolocation

In [154]:
geolocation = ["longitude", "latitude"]

sparkTimeStart = time.time()
avgHouseholdDF = df.groupBy(geolocation).agg({"households": "avg"})
sparkTimeEnd = time.time()
sparkTime = sparkTimeEnd - sparkTimeStart

avgHouseholdDF.show()

+---------+--------+-----------------+
|longitude|latitude|  avg(households)|
+---------+--------+-----------------+
|  -116.09|   34.15|           1482.0|
|  -116.31|   33.66|            713.0|
|   -116.9|   33.22|            703.0|
|  -116.96|   32.86|            503.0|
|  -116.97|   32.76|            406.0|
|  -116.99|   33.77|           1689.0|
|  -117.03|   33.18|            830.0|
|  -117.19|   33.69|           1074.0|
|   -117.2|   34.24|            423.0|
|  -117.54|   33.82|             28.0|
|  -117.24|   33.77|           1290.0|
|  -117.35|   33.69|            190.0|
|  -117.51|   34.14|            327.0|
|   -117.6|   33.45|            435.0|
|  -117.61|   33.42|            775.0|
|  -117.69|    33.6|390.6666666666667|
|  -117.73|   34.07|498.3333333333333|
|  -117.82|    33.8|            595.0|
|  -117.96|   33.71|            518.0|
|   -118.0|   34.15|            648.0|
+---------+--------+-----------------+
only showing top 20 rows



## 2.) Compare time for average: Spark vs Pandas

In [155]:
# Convert to Pandas
dfpd = df.toPandas()

pandasTimeStart = time.time()
avgHouseholdPandas = dfpd.groupby(geolocation)["households"].mean()
pandasTimeEnd = time.time()
pandasTime = pandasTimeEnd - pandasTimeStart

print("Pandas Avg Household by Geolocation:")
print(avgHouseholdPandas.head())

Pandas Avg Household by Geolocation:
longitude  latitude
-124.35    40.54       270.0
-124.30    41.80       478.0
           41.84       456.0
-124.27    40.69       465.0
-124.26    40.58       369.0
Name: households, dtype: float64


In [156]:
# compare times
print("Time taken by spark: ", sparkTime)
print("Time taken by pandas: ", pandasTime)

Time taken by spark:  0.022365570068359375
Time taken by pandas:  0.006831169128417969


Spark takes more time than pandas for grouping and aggregation.

## 3.) Highest median income by geolocation

In [157]:
sparkTimeStart = time.time()
maxMedianIncomeDF = df.groupBy(geolocation).agg({"median_income": "max"})
maxMedianIncomeDF.orderBy('max(median_income)', ascending = False).show(1)
sparkTimeEnd = time.time()
sparkTime = sparkTimeEnd - sparkTimeStart

+---------+--------+------------------+
|longitude|latitude|max(median_income)|
+---------+--------+------------------+
|  -118.41|   34.07|           15.0001|
+---------+--------+------------------+
only showing top 1 row



## 4.) Compare time for highest median income: Spark vs Pandas

In [158]:
# Convert to Pandas
dfpd = df.toPandas()

pandasTimeStart = time.time()
maxMedianIncomePandas = dfpd.groupby(geolocation)["median_income"].max()
maxMedianIncomePandas.sort_values(ascending=False).head(1)
pandasTimeEnd = time.time()
pandasTime = pandasTimeEnd - pandasTimeStart

In [159]:
# compare times
print("Time taken by spark: ", sparkTime)
print("Time taken by pandas: ", pandasTime)

Time taken by spark:  0.49741315841674805
Time taken by pandas:  0.005902290344238281


Spark takes more time than pandas for grouping, aggregation and sorting.

## 5.) Average median_house_value for different housing_median_age groups

In [160]:
avgMedianHouseValDF = df.groupBy("housing_median_age").agg({"median_house_value": "avg"})
avgMedianHouseValDF.orderBy("housing_median_age").show()

+------------------+-----------------------+
|housing_median_age|avg(median_house_value)|
+------------------+-----------------------+
|               1.0|               190250.0|
|               2.0|     229438.83673469388|
|               3.0|     239450.04347826086|
|               4.0|      230054.1055900621|
|               5.0|     211035.70854271358|
|               6.0|     206768.24031007753|
|               7.0|       188445.059602649|
|               8.0|     190805.07303370786|
|               9.0|      190306.9941860465|
|              10.0|     178416.39380530972|
|              11.0|            182480.3125|
|              12.0|          181590.640625|
|              13.0|     188065.47389558234|
|              14.0|     191181.28818443805|
|              15.0|     181031.02403846153|
|              16.0|     200354.50708661418|
|              17.0|     191772.24479166666|
|              18.0|     192074.71548117156|
|              19.0|     196017.75242718446|
|         

## 6.) Highest Population by location

In [161]:
populationDF = df.groupBy(geolocation).agg({"population": "max"})
populationDF.orderBy('max(population)', ascending = False).show(1)

+---------+--------+---------------+
|longitude|latitude|max(population)|
+---------+--------+---------------+
|  -117.42|   33.35|        35682.0|
+---------+--------+---------------+
only showing top 1 row



## 7.)  `total_bedrooms` to `total_rooms` ratio

In [162]:
# Calculate ratio
df_with_ratio = df.withColumn("bedroom_room_ratio", col("total_bedrooms") / col("total_rooms"))
df_with_ratio.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value| bedroom_room_ratio|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|0.22861724875267284|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|0.24849673202614378|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|0.24166666666666667|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|0.22451698867421718|
|  -11

In [163]:
# Calculate avg
average_ratio = df_with_ratio.agg(avg("bedroom_room_ratio")).collect()[0][0]
print(f"The average ratio of total_bedrooms to total_rooms is: {average_ratio}")

The average ratio of total_bedrooms to total_rooms is: 0.2127599882103483


## 8.) Correlation between `median_income` and `median_house_value`

In [164]:
correlation = df.select(corr("median_income", "median_house_value")).collect()[0][0]

print(f"The correlation between median_income and median_house_value is: {correlation}")

The correlation between median_income and median_house_value is: 0.6918706037806676
